In [20]:
from pathlib import Path
from typing import Dict, List, Tuple
import time
import pandas as pd
from ortools.linear_solver import pywraplp

ROOT = Path("..").resolve()
DATA_DIR = ROOT / "data"

ORLIB_DIR = DATA_DIR / "raw"
OPT_FILE  = ORLIB_DIR / "uncapopt.txt"
print("ORLIB_DIR:", ORLIB_DIR)
print("ORLIB_DIR exists:", ORLIB_DIR.exists())
print("cap71 exists:", (ORLIB_DIR / "cap71.txt").exists())
print("OPT_FILE:", OPT_FILE, "exists:", OPT_FILE.exists())

ORLIB_DIR: D:\aui\Ayoub\Spring_2026\AI for Digital Transformation\Project\Milestone 0 - Proposal\drafts\Repo\data\raw
ORLIB_DIR exists: True
cap71 exists: True
OPT_FILE: D:\aui\Ayoub\Spring_2026\AI for Digital Transformation\Project\Milestone 0 - Proposal\drafts\Repo\data\raw\uncapopt.txt exists: True


In [21]:
instances = ["cap71", "cap101", "cap131", "capa"]

for name in instances:
    p = ORLIB_DIR / f"{name}.txt"
    if not p.exists():
        print(name, "MISSING:", p)
        continue
    head = p.read_text()[:200].replace("\n", " ")
    print(name, "HEAD:", head)

cap71 HEAD:  16 50   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 0.   58268 7500.   58268 7500.   58268 7500. 
cap101 HEAD:  25 50   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 0.   58268 7500.   58268 7500.   58268 7500. 
cap131 HEAD:  50 50   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 7500.   58268 750
capa HEAD:  100 1000   capacity 2141200.   capacity 2107403.   capacity 1840969.   capacity 2217151.   capacity 2164343.   capacity 2158091.   capacity 2223692.   capacity 2113995.   capacity 1669402.   capacity


In [22]:
# Find uncapopt.txt (direct or recursive)
opt_candidates = list(DATA_DIR.glob("uncapopt.txt")) + list(DATA_DIR.rglob("uncapopt.txt"))
OPT_FILE = opt_candidates[0] if opt_candidates else None

print("OPT_FILE:", OPT_FILE)
print("OPT_FILE exists:", OPT_FILE.exists() if OPT_FILE else False)

OPT_FILE: D:\aui\Ayoub\Spring_2026\AI for Digital Transformation\Project\Milestone 0 - Proposal\drafts\Repo\data\raw\uncapopt.txt
OPT_FILE exists: True


In [23]:
def parse_orlib_uncap(path: str) -> Tuple[int, int, List[float], List[List[float]]]:
    tokens = Path(path).read_text().split()
    it = iter(tokens)

    m = int(next(it))
    n = int(next(it))

    fixed_costs: List[float] = []
    for _ in range(m):
        _capacity = float(next(it))  # ignored
        f_i = float(next(it))
        fixed_costs.append(f_i)

    costs: List[List[float]] = []
    for _ in range(n):
        _demand = float(next(it))  # ignored
        row = [float(next(it)) for _ in range(m)]
        costs.append(row)

    return m, n, fixed_costs, costs


def parse_uncapopt(path: str) -> Dict[str, float]:
    lines = Path(path).read_text().strip().splitlines()
    out: Dict[str, float] = {}
    for line in lines[1:]:
        parts = line.split()
        if len(parts) >= 2:
            out[parts[0]] = float(parts[1])
    return out

In [24]:
def solve_uflp_cbc(m: int, n: int, fixed_costs: List[float], costs: List[List[float]]) -> float:
    solver = pywraplp.Solver.CreateSolver("CBC")
    if solver is None:
        raise RuntimeError("CBC solver not available in this OR-Tools build.")

    y = [solver.BoolVar(f"y[{i}]") for i in range(m)]
    x = [[solver.BoolVar(f"x[{j},{i}]") for i in range(m)] for j in range(n)]

    for j in range(n):
        solver.Add(sum(x[j][i] for i in range(m)) == 1)

    for j in range(n):
        for i in range(m):
            solver.Add(x[j][i] <= y[i])

    obj = solver.Objective()
    for i in range(m):
        obj.SetCoefficient(y[i], fixed_costs[i])
    for j in range(n):
        for i in range(m):
            obj.SetCoefficient(x[j][i], costs[j][i])
    obj.SetMinimization()

    status = solver.Solve()
    if status not in (pywraplp.Solver.OPTIMAL, pywraplp.Solver.FEASIBLE):
        raise RuntimeError(f"Solver failed. Status={status}")

    return obj.Value()

In [25]:
opt_map = parse_uncapopt(str(OPT_FILE))
print("uncapopt entries:", len(opt_map))
print("cap71 best known:", opt_map.get("cap71"))

inst_path = ORLIB_DIR / "cap71.txt"
m, n, fixed_costs, costs = parse_orlib_uncap(str(inst_path))

print(f"Loaded instance: {inst_path.name} (m={m}, n={n})")

t0 = time.time()
val = solve_uflp_cbc(m, n, fixed_costs, costs)
runtime = time.time() - t0

best_known = opt_map.get("cap71")
gap = (val - best_known) / max(1.0, abs(best_known)) * 100.0

print(f"Solver objective: {val:.3f}")
print(f"Best known (uncapopt): {best_known:.3f}")
print(f"Gap vs best known: {gap:.6f}%")
print(f"Runtime: {runtime:.3f} sec")

uncapopt entries: 15
cap71 best known: 932615.75
Loaded instance: cap71.txt (m=16, n=50)
Solver objective: 932615.750
Best known (uncapopt): 932615.750
Gap vs best known: 0.000000%
Runtime: 0.054 sec


In [26]:
instances = ["cap71", "cap101", "cap131", "capa"]

rows = []
for name in instances:
    inst_path = ORLIB_DIR / f"{name}.txt"
    if not inst_path.exists():
        print(f"[SKIP] Missing: {name}")
        continue

    # quick format check: first ~200 chars must not include letters
    head = inst_path.read_text()[:200]
    if any(ch.isalpha() for ch in head):
        print(f"[SKIP] Non-raw format detected in {name}: contains letters in header")
        continue

    m, n, fixed_costs, costs = parse_orlib_uncap(str(inst_path))

    t0 = time.time()
    val = solve_uflp_cbc(m, n, fixed_costs, costs)
    runtime = time.time() - t0

    best = opt_map.get(name)
    gap = None if best is None else (val - best) / max(1.0, abs(best)) * 100.0

    rows.append({
        "instance": name,
        "m": m,
        "n": n,
        "objective": round(val, 3),
        "best_known": None if best is None else round(best, 3),
        "gap_%": None if gap is None else round(gap, 6),
        "runtime_sec": round(runtime, 3),
    })

df = pd.DataFrame(rows)
df

[SKIP] Non-raw format detected in capa: contains letters in header


,instance,m,n,objective,best_known,gap_%,runtime_sec
0,cap71,16,50,932615.750,932615.750,0.0,0.057
1,cap101,25,50,796648.438,796648.437,0.0,0.084
2,cap131,50,50,793439.563,793439.562,0.0,0.196
